In [1]:
import requests
import zipfile
import pandas as pd

In [2]:
def busca_informes_cvm(ano, mes):
  url = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'
  if ano < 2021 :
    url = url + 'HIST/'
    file_name = 'inf_diario_fi_{:02d}.zip'.format(ano)
  else :
    file_name = 'inf_diario_fi_{:02d}{:02d}.zip'.format(ano,mes)
  
  download = requests.get(url+file_name)
  with open(file_name, "wb") as arquivo_cvm:
    arquivo_cvm.write(download.content)
  arquivo_zip = zipfile.ZipFile(file_name)
  dados = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ";", encoding = 'ISO-8859-1')

  import os
  os.remove(file_name)

  return dados

def busca_cadastro_cvm(): 
  url = 'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv'
  dados = pd.read_csv(url, sep=';', encoding='ISO-8859-1',
                      usecols=['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL', 
                               'SIT','CLASSE', 'RENTAB_FUNDO', 'CONDOM', 
                               'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_PERFM',
                               'INF_TAXA_PERFM', 'TAXA_ADM', 'INF_TAXA_ADM', 
                               'CLASSE_ANBIMA'],low_memory=False)
  dados = dados[dados['SIT']=='EM FUNCIONAMENTO NORMAL']
  return dados


In [3]:
dados_fundo = busca_informes_cvm(ano=2024, mes=2)
print(*dados_fundo.columns)
dados_cadastro = busca_cadastro_cvm()
print(*dados_cadastro.columns)

TP_FUNDO CNPJ_FUNDO DT_COMPTC VL_TOTAL VL_QUOTA VL_PATRIM_LIQ CAPTC_DIA RESG_DIA NR_COTST
TP_FUNDO CNPJ_FUNDO DENOM_SOCIAL SIT CLASSE RENTAB_FUNDO CONDOM TRIB_LPRAZO PUBLICO_ALVO TAXA_PERFM INF_TAXA_PERFM TAXA_ADM INF_TAXA_ADM CLASSE_ANBIMA


### rentabilidade no mês

In [4]:
## só o 1o e ultimo dia interessam

data_inicio_mes = (dados_fundo['DT_COMPTC'].sort_values(ascending = True).unique())[0]
data_fim_mes = (dados_fundo['DT_COMPTC'].sort_values(ascending = True).unique())[-1]
dados_fundos_filtrado = dados_fundo[(dados_fundo['DT_COMPTC'].isin([data_inicio_mes, data_fim_mes]))]


In [5]:

fundos_cnpj = dados_fundos_filtrado.pivot(index='DT_COMPTC', columns=['CNPJ_FUNDO'] ) 
cotas_normalizadas = fundos_cnpj['VL_QUOTA'] / fundos_cnpj['VL_QUOTA'].iloc[0]
rentabilidade = (((cotas_normalizadas.iloc[-1] - 1)*100).round(2)).to_frame(name='rendeu').reset_index()

In [6]:
base = pd.merge(dados_fundos_filtrado, dados_cadastro.drop(columns='TP_FUNDO'), how = "left",
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])
base_final = pd.merge(base,rentabilidade, how = "left",
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

base_final.columns

Index(['TP_FUNDO', 'CNPJ_FUNDO', 'DT_COMPTC', 'VL_TOTAL', 'VL_QUOTA',
       'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST', 'DENOM_SOCIAL',
       'SIT', 'CLASSE', 'RENTAB_FUNDO', 'CONDOM', 'TRIB_LPRAZO',
       'PUBLICO_ALVO', 'TAXA_PERFM', 'INF_TAXA_PERFM', 'TAXA_ADM',
       'INF_TAXA_ADM', 'CLASSE_ANBIMA', 'rendeu'],
      dtype='object')

Rendimento de um fundo específico

In [7]:
busca_fundo = base_final[base_final['DENOM_SOCIAL'].str.contains("DYNAMO", na = False)]

from IPython.display import display, HTML
display(HTML(busca_fundo[['CNPJ_FUNDO' ,  'DENOM_SOCIAL']].drop_duplicates().to_html(index=False)))

CNPJ_FUNDO,DENOM_SOCIAL
35.002.734/0001-94,DYNAMO PLUS FUNDO DE INVESTIMENTO DE ACOES
37.916.879/0001-26,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM AÇÕES
73.232.530/0001-39,DYNAMO COUGAR FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES


In [8]:
cnpj = "73.232.530/0001-39"
FI = busca_fundo[(busca_fundo['CNPJ_FUNDO'] == cnpj)][['DENOM_SOCIAL', 'VL_PATRIM_LIQ','rendeu']].iloc[-1].values

print(FI[0])
print(f'Patrimônio R$ {(FI[1]/1000000).round(2)} MM') 
print(f'Retorno no mês {FI[2]}%')

DYNAMO COUGAR FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES
Patrimônio R$ 6250.62 MM
Retorno no mês 3.32%


#### melhores e piores fundos

In [9]:
# mostra os melhores que satifazem as restrições
minimo_cotistas = 5000
top = 50
# classe = 'Fundo de Renda Fixa'
classe = 'Fundo de Ações'
# classe = 'Fundo Multimercado'
'''
[nan, 'Fundo de Renda Fixa', 'Fundo de Ações', 'Fundo Multimercado',
       'Fundo Cambial', 'FMP-FGTS', 'FIDC', 'FIDC-NP', 'FIC FIDC',
       'FICFIDC-NP', 'FIDCFIAGRO', 'FII', 'FII-FIAGRO', 'FIP', 'FIP EE',
       'FIP Multi', 'FIP CS', 'FIP-FIAGRO', 'FIC FIP', 'FIP IE',
       'FUNCINE']
'''       

filtro = (base_final['SIT'] == 'EM FUNCIONAMENTO NORMAL')&\
(base_final['NR_COTST'] >= minimo_cotistas)&\
(base_final['CLASSE'] == classe)

infos = ['VL_PATRIM_LIQ', 'NR_COTST', 'DENOM_SOCIAL',
        'PUBLICO_ALVO', 'TAXA_PERFM', 'TAXA_ADM',
        'CLASSE_ANBIMA', 'rendeu']

melhores

In [10]:
# base_final[filtro].sort_values(by='rendeu',ascending=False)[infos]
display(HTML(base_final[filtro].sort_values(by='rendeu',ascending=False)[infos].head(top).to_html(index=False)))

VL_PATRIM_LIQ,NR_COTST,DENOM_SOCIAL,PUBLICO_ALVO,TAXA_PERFM,TAXA_ADM,CLASSE_ANBIMA,rendeu
7.073353e+08,7465,MS GLOBAL OPPORTUNITIES DÓLAR ADVISORY FIC FIA IE,Qualificado,0.0,0.80,Ações Invest. no Exterior,8.74
7.632175e+08,7314,MS GLOBAL OPPORTUNITIES DÓLAR ADVISORY FIC FIA IE,Qualificado,0.0,0.80,Ações Invest. no Exterior,8.74
4.539868e+07,9596,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Público Geral,0.0,1.50,Fundos de Mono Ação,7.50
4.216057e+07,9807,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Público Geral,0.0,1.50,Fundos de Mono Ação,7.50
4.979576e+08,16889,ITAÚ INDEX US TECH FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES,Público Geral,0.0,0.80,Ações Livre,7.47
5.809962e+08,17752,ITAÚ INDEX US TECH FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES,Público Geral,0.0,0.80,Ações Livre,7.47
4.970856e+07,9170,TREND BOLSA CHINESA FUNDO DE INVESTIMENTO EM AÇÕES,Público Geral,0.0,0.70,Ações Livre,7.03
4.779360e+07,9423,TREND BOLSA CHINESA FUNDO DE INVESTIMENTO EM AÇÕES,Público Geral,0.0,0.70,Ações Livre,7.03
4.949491e+08,16157,BB AÇÕES TECNOLOGIA BDR NÍVEL I FUNDO DE INVESTIMENTO,Público Geral,20.0,1.90,Ações Setoriais,6.54
5.285061e+08,17010,BB AÇÕES TECNOLOGIA BDR NÍVEL I FUNDO DE INVESTIMENTO,Público Geral,20.0,1.90,Ações Setoriais,6.54


piores

In [11]:
display(HTML(base_final[filtro].sort_values(by='rendeu',ascending=True)[infos].head(top).to_html(index=False)))

VL_PATRIM_LIQ,NR_COTST,DENOM_SOCIAL,PUBLICO_ALVO,TAXA_PERFM,TAXA_ADM,CLASSE_ANBIMA,rendeu
6.972235e+07,6695,TREND XP INC FI EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR NÍVEL I,Público Geral,0.0,0.20,Ações Livre,-5.67
6.481225e+07,6697,TREND XP INC FI EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR NÍVEL I,Público Geral,0.0,0.20,Ações Livre,-5.67
8.500780e+08,15913,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA PETROBRAS,Público Geral,0.0,1.50,Fundos de Mono Ação,-4.10
8.950232e+08,15793,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA PETROBRAS,Público Geral,0.0,1.50,Fundos de Mono Ação,-4.10
2.734597e+08,5555,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES PETROBRAS,Público Geral,0.0,1.50,Fundos de Mono Ação,-4.08
2.865949e+08,5588,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES PETROBRAS,Público Geral,0.0,1.50,Fundos de Mono Ação,-4.08
3.811067e+08,12828,ITAÚ AÇÕES PETROBRÁS - FUNDO DE INVESTIMENTO,Público Geral,0.0,0.80,Fundos de Mono Ação,-4.04
3.601025e+08,12713,ITAÚ AÇÕES PETROBRÁS - FUNDO DE INVESTIMENTO,Público Geral,0.0,0.80,Fundos de Mono Ação,-4.04
2.109501e+08,5953,CSN INVEST FUNDO DE INVESTIMENTO EM AÇÕES,Público Geral,0.0,0.85,Ações Livre,-4.04
2.018154e+08,5945,CSN INVEST FUNDO DE INVESTIMENTO EM AÇÕES,Público Geral,0.0,0.85,Ações Livre,-4.04


rank por palavra chave no nome

In [12]:
palavra = 'quant'
classe = 'Fundo de Ações'

filtro = (base_final['SIT'] == 'EM FUNCIONAMENTO NORMAL')&\
(base_final['CLASSE'] == classe)&\
(base_final['DENOM_SOCIAL'].str.contains(palavra.upper()))

display(HTML(base_final[filtro].sort_values(by='rendeu',ascending=False)[infos].head(top).to_html(index=False)))

VL_PATRIM_LIQ,NR_COTST,DENOM_SOCIAL,PUBLICO_ALVO,TAXA_PERFM,TAXA_ADM,CLASSE_ANBIMA,rendeu
5.018577e+07,10,BRADESCO QUANT INSTITUCIONAL GLOBAL FUNDO DE INVESTIMENTO EM AÇÕES USD,Qualificado,NaN,NaN,Ações Livre,4.95
5.253267e+07,10,BRADESCO QUANT INSTITUCIONAL GLOBAL FUNDO DE INVESTIMENTO EM AÇÕES USD,Qualificado,NaN,NaN,Ações Livre,4.95
1.453665e+06,1,PROSPERA QUANT FUNDO DE INVESTIMENTO FINANCEIRO,Público Geral,NaN,NaN,Ações Livre,4.39
1.392531e+06,1,PROSPERA QUANT FUNDO DE INVESTIMENTO FINANCEIRO,Público Geral,NaN,NaN,Ações Livre,4.39
7.711962e+07,6,FUNDO DE INVESTIMENTO EM ACOES CAIXA PREVINVEST AÇÕES LIVRE QUANTITATIVO PREVIDENCIÁRIO,Profissional,0.0,0.0,Previdência Ações Ativo,2.33
8.347392e+07,6,FUNDO DE INVESTIMENTO EM ACOES CAIXA PREVINVEST AÇÕES LIVRE QUANTITATIVO PREVIDENCIÁRIO,Profissional,0.0,0.0,Previdência Ações Ativo,2.33
8.144444e+08,5,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA MASTER AÇÕES LIVRE QUANTITATIVO,Público Geral,0.0,0.0,Ações Livre,1.81
8.154527e+08,5,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA MASTER AÇÕES LIVRE QUANTITATIVO,Público Geral,0.0,0.0,Ações Livre,1.81
5.905752e+06,3,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA MASTER DIVIDENDOS QUANTITATIVO,Público Geral,NaN,NaN,Ações Dividendos,1.71
5.574104e+06,3,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA MASTER DIVIDENDOS QUANTITATIVO,Público Geral,NaN,NaN,Ações Dividendos,1.71
